In [2]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

In [3]:
# import pdb file
pdb = PDBFile('Pdb_files/FUS_withH.pdb')

In [4]:
# Get the positions of all the atoms in the pdb file. 
positions = pdb.getPositions()
print(positions[0])
print(positions[0][2])   

Vec3(x=-6.126100000000001, y=-3.2746000000000004, z=-0.0601) nm
-0.0601 nm


In [5]:
# Remove the units. 
#positions = pdb.getPositions()/nanometer
#print(positions[0])
#print(positions[0][0])

In [6]:
# Converting into different units. 
#positions = pdb.getPositions()/angstrom        
#print(positions[0])
#print(positions[0][0])

In [7]:
for pos in positions[:5]: print(pos)

Vec3(x=-6.126100000000001, y=-3.2746000000000004, z=-0.0601) nm
Vec3(x=-5.9834000000000005, y=-3.3064999999999998, z=-0.0811) nm
Vec3(x=-5.907500000000001, y=-3.1993, z=-0.0031000000000000003) nm
Vec3(x=-5.95, y=-3.3059, z=-0.2318) nm
Vec3(x=-5.912800000000001, y=-3.0855, z=-0.046400000000000004) nm


In [8]:
top = pdb.getTopology()
print(top)
print(top.getNumAtoms())
print(top.getNumBonds())

<Topology; 1 chains, 526 residues, 7093 atoms, 7177 bonds>
7093
7177


In [9]:
# Get the bonds and know the atom number it is bonded to. 
bonds = list(top.bonds())
top_five_bonds = []
for bond in bonds[:5]: top_five_bonds.append(bond); print(bond)

print(top_five_bonds[0])
print(top_five_bonds[0].atom1, top_five_bonds[0].atom2)
print(top_five_bonds[0].atom1.name, top_five_bonds[0].atom2.name)
print(top_five_bonds[0].atom1.id, top_five_bonds[0].atom2.id)
print(top_five_bonds[0].atom1.index, top_five_bonds[0].atom2.index)

print(top_five_bonds[4])
print(top_five_bonds[4].atom1, top_five_bonds[4].atom2)
print(top_five_bonds[4].atom1.name, top_five_bonds[4].atom2.name)
print(top_five_bonds[4].atom1.id, top_five_bonds[4].atom2.id)           # id matches with the pdb file id. 
print(top_five_bonds[4].atom1.index, top_five_bonds[4].atom2.index)     # index is always less than 1 of id. 

Bond(<Atom 2 (C) of chain 0 residue 0 (MET)>, <Atom 1 (CA) of chain 0 residue 0 (MET)>)
Bond(<Atom 2 (C) of chain 0 residue 0 (MET)>, <Atom 4 (O) of chain 0 residue 0 (MET)>)
Bond(<Atom 1 (CA) of chain 0 residue 0 (MET)>, <Atom 3 (CB) of chain 0 residue 0 (MET)>)
Bond(<Atom 1 (CA) of chain 0 residue 0 (MET)>, <Atom 11 (HA) of chain 0 residue 0 (MET)>)
Bond(<Atom 1 (CA) of chain 0 residue 0 (MET)>, <Atom 0 (N) of chain 0 residue 0 (MET)>)
Bond(<Atom 2 (C) of chain 0 residue 0 (MET)>, <Atom 1 (CA) of chain 0 residue 0 (MET)>)
<Atom 2 (C) of chain 0 residue 0 (MET)> <Atom 1 (CA) of chain 0 residue 0 (MET)>
C CA
3 2
2 1
Bond(<Atom 1 (CA) of chain 0 residue 0 (MET)>, <Atom 0 (N) of chain 0 residue 0 (MET)>)
<Atom 1 (CA) of chain 0 residue 0 (MET)> <Atom 0 (N) of chain 0 residue 0 (MET)>
CA N
2 1
1 0


In [10]:
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff, nonbondedCutoff=1*nanometer, constraints=HBonds)

In [11]:
system.getForces()

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x10f17a6f0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x10f17a7e0> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x10f17a870> >,
 <openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x10f17a900> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x10f17a990> >]

In [12]:
# Getting the NonbondedForce
print(system.getForces()[1])
nb_forces = system.getForces()[1]
print(nb_forces.getNumParticles())
print(nb_forces.getCutoffDistance())

<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x10f17aa20> >
7093
1.0 nm


In [13]:
# Setting simulation object
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)

# set the initial position of the simulation.
simulation.context.setPositions(pdb.positions)

state = simulation.context.getState(getEnergy=True)
print(state.getPotentialEnergy())

-13227.516967773438 kJ/mol


In [14]:
# I want to change the parameters of the non-bonded force and double it.
nb_particles = system.getNumParticles()
sigma = []
epsilon = []
charge = []

# Getting the parameter of the non-bonded force.
for n in range(nb_particles):
    param = nb_forces.getParticleParameters(n)
    charge.append(param[0])
    sigma.append(param[1])
    epsilon.append(param[2])

print(param)

# Double the parameter. 
for i in range(nb_particles):
    nb_forces.setParticleParameters(i, charge[i], sigma[i]*0.97, epsilon[i])


[Quantity(value=0.4017, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]


In [15]:
# update the context 
state = simulation.context.getState(getEnergy=True)
print(state.getPotentialEnergy())

-13227.516967773438 kJ/mol
